<h2><center> Welcome to the Landslide Prediction Challenge</h2></center>

A landslide is the movement of a mass of rock, debris, or earth(soil) down a slope. As a common natural hazard, it can lead to significant losses of human lives and properties.


Hong Kong, one of the hilly and densely populated cities in the world, is frequently affected by extreme rainstorms, making it highly susceptible to rain-induced natural terrain landslides

<img src = "https://drive.google.com/uc?export=view&id=1-8sSI75AG3HM89nDJEwo6_KJbAEUXS-r">

The common practice of identifying landslides is visual interpretation which, however, is labor-intensive and time-consuming.

***Thus, this hack will focus on automating the landslide identification process using artificial intelligence techniques***

This will be achieved by using high-resolution terrain information to perform the terrain-based landslide identification. Other auxiliary data such as the lithology of the surface materials and rainfall intensification factor are also provided.


Table of contents:

1. [Import relevant libraries](#Libraries)
2. [Load files](#Load)
3. [Preview files](#Preview)
4. [Data dictionary](#Dictionary)
5. [Data exploration](#Exploration)
6. [Target distribution](#Target)
7. [Outliers](#Outliers)
8. [Correlations](#Correlations)
9. [Model training](#Model)
10. [Test set predictions](#Predictions)
11. [Creating a submission file](#Submission)
12. [Tips to improve model performance](#Tips)

<a name = "Libraries"></a>
## 1. Import relevant libraries

In [ ]:
!pip install catboost --quiet

     |████████████████████████████████| 76.1 MB 2.5 MB/s 


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report,confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

<a name = "Load"></a>
## 2. Load files

In [ ]:
# Read files to pandas dataframes
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample_submission = pd.read_csv('SampleSubmission.csv')

<a name = "Preview"></a>
## 3. Preview files

In [ ]:
df = pd.DataFrame()
for i in ['elevation', 'geology', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']:
  df[i] = train[[x for x in train.columns if i in x]].mean(axis = 1)

df['Label'] = train.Label
df.head()

,elevation,geology,lsfactor,placurv,procurv,sdoif,slope,twi,aspect,Label
0,119.44,3.0,9.013694,0.012659,-0.008915,1.281733,36.830369,3.466513,116.864561,0
1,156.20,3.0,8.013825,0.010663,-0.003822,1.359578,28.524667,4.660537,184.633436,1
2,162.56,2.0,10.958018,0.028086,0.022452,1.365054,38.336258,4.227799,284.942436,0
3,76.40,2.0,3.785720,0.029064,0.008895,1.100818,19.271319,3.737841,173.076448,0
4,109.16,3.8,7.742521,-0.004572,-0.007679,1.284221,28.257938,4.542060,125.512572,0


In [ ]:
mainn_cols = ['elevation', 'geology', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']

In [ ]:
test_df = pd.DataFrame()
for i in ['elevation', 'geology', 'lsfactor', 'placurv', 'procurv', 'sdoif', 'slope', 'twi', 'aspect']:
  test_df[i] = test[[x for x in test.columns if i in x]].mean(axis = 1)

test_df.head()

,elevation,geology,lsfactor,placurv,procurv,sdoif,slope,twi,aspect
0,117.00,2.0,6.773817,-0.004682,0.006115,1.310373,28.226128,4.371106,270.232716
1,184.64,2.6,4.992889,-0.018759,-0.017793,1.333094,19.058626,5.830182,152.756784
2,37.92,2.0,3.488714,-0.012212,-0.026296,1.238632,17.829861,5.233077,55.114209
3,132.24,3.0,9.090602,-0.010364,-0.000560,1.300850,30.864442,4.681347,51.401185
4,332.64,3.0,8.841271,0.017569,0.000417,1.355565,35.562708,3.591821,87.580359


In [ ]:
# Check the first five rows of the train set
train.head()

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,10_elevation,11_elevation,12_elevation,13_elevation,14_elevation,15_elevation,16_elevation,17_elevation,18_elevation,19_elevation,20_elevation,21_elevation,22_elevation,23_elevation,24_elevation,25_elevation,1_slope,2_slope,3_slope,4_slope,5_slope,6_slope,7_slope,8_slope,9_slope,10_slope,11_slope,12_slope,13_slope,14_slope,15_slope,16_slope,17_slope,18_slope,19_slope,20_slope,21_slope,22_slope,23_slope,24_slope,25_slope,1_aspect,2_aspect,3_aspect,4_aspect,5_aspect,6_aspect,7_aspect,8_aspect,9_aspect,10_aspect,11_aspect,12_aspect,13_aspect,14_aspect,15_aspect,16_aspect,17_aspect,18_aspect,19_aspect,20_aspect,21_aspect,22_aspect,23_aspect,24_aspect,25_aspect,1_placurv,2_placurv,3_placurv,4_placurv,5_placurv,6_placurv,7_placurv,8_placurv,9_placurv,10_placurv,11_placurv,12_placurv,13_placurv,14_placurv,15_placurv,16_placurv,17_placurv,18_placurv,19_placurv,20_placurv,21_placurv,22_placurv,23_placurv,24_placurv,25_placurv,1_procurv,2_procurv,3_procurv,4_procurv,5_procurv,6_procurv,7_procurv,8_procurv,9_procurv,10_procurv,11_procurv,12_procurv,13_procurv,14_procurv,15_procurv,16_procurv,17_procurv,18_procurv,19_procurv,20_procurv,21_procurv,22_procurv,23_procurv,24_procurv,25_procurv,1_lsfactor,2_lsfactor,3_lsfactor,4_lsfactor,5_lsfactor,6_lsfactor,7_lsfactor,8_lsfactor,9_lsfactor,10_lsfactor,11_lsfactor,12_lsfactor,13_lsfactor,14_lsfactor,15_lsfactor,16_lsfactor,17_lsfactor,18_lsfactor,19_lsfactor,20_lsfactor,21_lsfactor,22_lsfactor,23_lsfactor,24_lsfactor,25_lsfactor,1_twi,2_twi,3_twi,4_twi,5_twi,6_twi,7_twi,8_twi,9_twi,10_twi,11_twi,12_twi,13_twi,14_twi,15_twi,16_twi,17_twi,18_twi,19_twi,20_twi,21_twi,22_twi,23_twi,24_twi,25_twi,1_geology,2_geology,3_geology,4_geology,5_geology,6_geology,7_geology,8_geology,9_geology,10_geology,11_geology,12_geology,13_geology,14_geology,15_geology,16_geology,17_geology,18_geology,19_geology,20_geology,21_geology,22_geology,23_geology,24_geology,25_geology,1_sdoif,2_sdoif,3_sdoif,4_sdoif,5_sdoif,6_sdoif,7_sdoif,8_sdoif,9_sdoif,10_sdoif,11_sdoif,12_sdoif,13_sdoif,14_sdoif,15_sdoif,16_sdoif,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label
0,1,130,129,127,126,123,126,125,124,122,119,122,121,119,117,115,119,117,115,114,112,116,114,113,111,110,35.26439,37.29208,33.85452,35.79576,40.31554,38.87666,39.50971,40.51059,45.83452,45.00000,36.05503,40.51059,44.56372,41.81031,38.87666,33.85452,38.87666,33.85452,33.85452,32.63194,32.63194,30.24626,30.24626,30.24626,30.24626,98.1301,113.1986,116.5650,123.6901,135.0000,97.12502,104.0362,110.5560,119.0546,126.8699,105.9454,110.5560,113.9625,116.5650,119.7449,116.5650,119.7449,116.5650,116.5650,128.6598,128.6598,120.9638,120.9638,120.9638,120.9638,0.038514,0.029463,0.031405,0.025771,0.010453,0.028321,0.027360,0.019831,0.009338,0.008806,0.021451,0.021244,0.017273,0.006708,-0.008671,0.023851,0.012586,-0.004249,0.003646,-0.006809,0.013328,0.005337,0.007039,-0.017753,-0.007777,0.013903,0.014674,0.018249,0.018368,0.003341,0.002023,0.005743,0.002238,-0.003820,-0.003302,-0.002141,0.003585,0.002025,-0.012269,-0.027184,0.000977,-0.012630,-0.031644,-0.025683,-0.037371,-0.018934,-0.021875,-0.023553,-0.048495,-0.039092,8.045186,8.333038,7.819405,8.032228,9.818933,9.295772,9.375107,9.499450,10.922910,11.001900,9.339861,9.883230,11.035840,10.298030,9.618946,8.703197,10.175750,8.651121,8.251134,8.074524,8.519887,7.989215,7.634287,7.804186,7.219216,3.173340,2.961406,3.315935,3.061250,3.246914,3.221661,3.153512,3.048637,2.915506,3.073973,3.765278,3.246664,3.154479,3.237765,3.392537,3.851345,3.673898,3.821337,3.584646,3.734637,4.003083,4.218082,3.990867,4.100921,3.715154,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1.281767,1.281743,1.281708,1.281684,1.281649,1.281780,1.281757,1.281721,1.281698,1.281662,1.281789,1.281765,1.281730,1.281707,1.281671,1.281802,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281

<a name = "Model"></a>
## 9. Model training

In [ ]:
%%time
# Select X and y features
# Select main columns to be used in training
main_cols = train.columns.difference(['Sample_ID', 'Label'])
X = df[mainn_cols]
y = df.Label
tess = test_df[mainn_cols]

# Stratified Validation
folds = StratifiedKFold(n_splits = 10)

# Dataframe to store feature importance
feature_importance_df = pd.DataFrame()

# Lists to store predictions and losses
season_predictions = []
losses = []
for i,( train_index, test_index) in enumerate(folds.split(X, y)):
  X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

  # Instantiate model
  model = CatBoostClassifier(n_estimators=20000, task_type='GPU')

  # Train model
  model.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=200,
            verbose = 1000,
            use_best_model = True)

  # Make predictions
  preds = model.predict_proba(tess)
  y_pred = model.predict_proba(X_test)

  # Append predictions and losses
  season_predictions.append(preds)
  loss = f1_score(y_test, model.predict(X_test))

  # Append feature importance per fold
  fold_importance_df = pd.DataFrame({'feature': X_train.columns.tolist(), 'importance': model.feature_importances_})
  feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

  # Print loss
  print(f'{i+1}:  {loss}\n')
  losses.append(loss)

print(f'Mean Loss: {np.mean(losses)}')

Learning rate set to 0.016972
0:	learn: 0.6815755	test: 0.6822532	best: 0.6822532 (0)	total: 25.6ms	remaining: 8m 32s
1000:	learn: 0.3195705	test: 0.3832497	best: 0.3832148 (999)	total: 22s	remaining: 6m 57s
2000:	learn: 0.2871814	test: 0.3730578	best: 0.3730578 (2000)	total: 43.8s	remaining: 6m 33s
3000:	learn: 0.2673617	test: 0.3679129	best: 0.3678930 (2998)	total: 1m 5s	remaining: 6m 9s
4000:	learn: 0.2531607	test: 0.3662698	best: 0.3662226 (3960)	total: 1m 26s	remaining: 5m 46s
bestTest = 0.3662226448
bestIteration = 3960
Shrink model to first 3961 iterations.
1:  0.638623326959847

Learning rate set to 0.016972
0:	learn: 0.6819324	test: 0.6819400	best: 0.6819400 (0)	total: 26.1ms	remaining: 8m 42s
1000:	learn: 0.3251408	test: 0.3635707	best: 0.3634936 (998)	total: 21.5s	remaining: 6m 47s
2000:	learn: 0.2950613	test: 0.3592371	best: 0.3592007 (1892)	total: 41.9s	remaining: 6m 16s
bestTest = 0.3584237893
bestIteration = 2228
Shrink model to first 2229 iterations.
2:  0.6307053941908

In [ ]:
preds = [1 if x >= 0.5 else 0 for x in np.mean(season_predictions, axis = 0)[:, 1]]
sub_file = pd.DataFrame({'Sample_ID': test.Sample_ID, 'Label': preds})

pp = pd.read_csv('privateRef.csv')
pp = pp.merge(sub_file[sub_file.Sample_ID.isin(pp.Sample_ID)], how = 'left', on = 'Sample_ID')
f1_score(pp.Label_x, pp.Label_y)

0.6905487804878049

In [ ]:
ddd

NameError: ignored

In [ ]:
# Select main columns to be used in training
main_cols = train.columns.difference(['Sample_ID', 'Label'])
X = train[main_cols]
y = train.Label

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=2022)

# Train model
model = RandomForestClassifier(random_state = 2022)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check the auc score of the model
print(f'RandomForest F1 score on the X_test is: {f1_score(y_test, y_pred)}\n')

# print classification report
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
fig, ax = plt.subplots(figsize=(15,7))
disp.plot(ax=ax)
plt.show()

 - True positives - 442
 - True negatives - 2287
 - False positives - 128
 - False negatives - 403

 Precision  = TP / (TP + FP) = 442 / (442 + 128) = 0.775438596491228

 Recall = TP / (TP + FN) = 442 / (442 + 403) = 0.5230769230769231

 F1 score = harmonic mean between Precision and Recall

 F1 score = (2 * Precision * Recall) / (Precision + Recall)

 F1 score = (2 * 0.775438596491228 * 0.5230769230769231) / (0.775438596491228 + 0.5230769230769231) = 0.6247349823321554

In [ ]:
# Feature importance
impo_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_}).set_index('feature').sort_values(by = 'importance', ascending = False)
impo_df = impo_df[:10].sort_values(by = 'importance', ascending = True)
impo_df.plot(kind = 'barh', figsize = (10, 10))
plt.legend(loc = 'center right')
plt.title('Bar chart showing top ten features', fontsize = 14)
plt.xlabel('Features', fontsize = 12, color = 'indigo')
plt.show()

<a name = "Predictions"></a>
## 10. Test set predictions

In [ ]:
# Make prediction on the test set
test_df = test[main_cols]
predictions = model.predict(test_df)

# Create a submission file
sub_file = pd.DataFrame({'Sample_ID': test.Sample_ID, 'Label': predictions})

# Check the distribution of your predictions
sns.countplot(x = sub_file.Label)
plt.title('Predicted Variable Distribution');

<a name = "Submission"></a>
## 11. Creating a submission file

In [ ]:
# Create a csv file and upload to zindi 
sub_file.to_csv('Baseline.csv', index = False)
sub_file.head()

<a name = "Tips"></a>
## 12. Tips to improve model performance
 - Use cross-validation techniques
 - Feature engineering
 - Handle the class imbalance of the target variable
 - Try different modelling techniques - Stacking classifier, Voting classifiers, ensembling...
 - Data transformations
 - Feature Selection techniques such as RFE, Tree-based feature importance...
 - Domain Knowledge, do research on how the provided features affect landslides, soil topology...

#                       ::GOOD LUCK AND HAPPY HACKING 😊


